In [2]:
from Dataset import lerCSV,lerXLSX
import warnings
import sys 
import numpy as np
import pickle
import theanets
from sklearn import preprocessing
from random import shuffle
from sklearn.metrics import precision_score,recall_score
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from math import sqrt
import math
%matplotlib inline

In [3]:
warnings.filterwarnings("ignore")   
matrix = lerXLSX("Banco de Dados - Infarto treinoTeste.xlsx",training=True,agrupamento=False)
def sigmoid(x):
  x = (x - 0.5) *  10
  return 1 / (1 + math.exp(-x))
def pesoUm(line):    
    return pesoFixo(line,0)
def pesoDois(line):    
    return pesoFixo(line,1)
def pesoTres(line):    
    return pesoFixo(line,2)
def pesoQuatro(line):    
    return pesoFixo(line,3)
def pesoOito(line):
    return pesoFixo(line,7)
def pesoFixo(line,peso):
    return int(line[3] == 1)*peso+1

In [4]:
k = 10
lista = list(range(0,len(matrix))) 
shuffle(lista)
a = np.asarray(lista) % k

In [5]:
min_max_scaler = preprocessing.MinMaxScaler()
matrixProcessed = min_max_scaler.fit_transform(matrix[:,:3])
#matrixProcessed = preprocessing.normalize(matrix[:,:3], norm='l2')
#matrixProcessed = np.array([[matrixProcessed[y,0],matrixProcessed[y,1],matrixProcessed[y,2],matrixProcessed[y,3]]  for y in range(0,len(matrix[:,0]))])
matrixProcessed = matrixProcessed = np.array([[matrixProcessed[y,0],sigmoid(matrixProcessed[y,1]),matrixProcessed[y,2]]  for y in range(0,len(matrix[:,0]))])


In [6]:
def specificity(outputs,predicts):
    neg = np.sum([output == 0 for output in outputs])
    tn = np.sum([predicts[x] == outputs[x] and outputs[x] == 0 for x in range(len(predicts))])
    return tn / neg

def KFold(matrix,metric):
    k = 8
    
    limit = 3
    peso = metric[2]
    
    lista = list(range(1,len(matrix))) 
    shuffle(lista)
    a = np.asarray(lista) % k   
    
    scores = np.zeros(k)
    recall = np.zeros(k)
    precision = np.zeros(k)
    average_precision = np.zeros(k)
    
    for fold in range(0,k):
        treinoIndex = np.where(a != fold)[0]
        testeIndex =  np.where(a == fold)[0]        
        
        inputs = np.array([[matrix[y,0],matrix[y,1],matrix[y,2]]  for y in treinoIndex])
        weights = np.array([peso(matrix[y,:]) for y in treinoIndex],dtype="int32")        
        outputs = np.array([int(matrix[y,3] == 1) for y in treinoIndex],dtype="int32")
            
        model = theanets.Classifier(metric[1],loss='CrossEntropy',weighted=True)  
        model.train([inputs, outputs,weights],algo='sgd',learning_rate=0.01,momentum=0.9)       
        
        testOutputs = [matrix[y,3] for y in testeIndex] 
        testInputs = np.array([matrix[y,:3]  for y in testeIndex])   
        predicts = model.predict(testInputs)
        scores[fold] = sum([int(predicts[x] == testOutputs[x]) for x in range(len(predicts))]) / len(predicts) 
        recall[fold] = recall_score(testOutputs, predicts)
        precision[fold] = specificity(testOutputs, predicts)        
        
    
    print("Final:"+ metric[0] + " = Score:%2.2e[+/- %2.2e]"%(np.mean(scores),np.std(scores)))
    print("Final:"+ metric[0] + " = Sensitive:%2.2e[+/- %2.2e]"%(np.mean(recall),np.std(recall)))
    print("Final:"+ metric[0] + " = specificity:%2.2e[+/- %2.2e]"%(np.mean(precision),np.std(precision)))       
    return np.mean(scores), np.std(scores)

In [ ]:
inputOutput = np.array([np.concatenate((matrixProcessed[x,:3],[matrix[x,3] == 1]),axis=0) for x in range(0,len(matrix[:,3]))])

In [ ]:
metrics = [ ["neural network (3,2)",[3,2],pesoUm], ["neural network (3,2)",[3,2],pesoDois],["neural network (3,2)",[3,2],pesoTres],["neural network (3,2)",[3,2],pesoQuatro],["neural network (3,2)",[3,2],pesoOito]]
for metric in metrics:
    KFold(inputOutput, metric=metric)


Final:neural network (3,2) = Score:7.59e-01[+/- 3.58e-02]
Final:neural network (3,2) = Sensitive:5.47e-01[+/- 9.82e-02]
Final:neural network (3,2) = specificity:8.75e-01[+/- 3.04e-02]
Final:neural network (3,2) = Score:7.29e-01[+/- 3.03e-02]
Final:neural network (3,2) = Sensitive:7.64e-01[+/- 9.32e-02]
Final:neural network (3,2) = specificity:7.05e-01[+/- 7.00e-02]
